<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#RegulonDB" data-toc-modified-id="RegulonDB-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>RegulonDB</a></span><ul class="toc-item"><li><span><a href="#Convert-RegulonDB-names-to-b-numbers" data-toc-modified-id="Convert-RegulonDB-names-to-b-numbers-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Convert RegulonDB names to b-numbers</a></span></li><li><span><a href="#Get-TF-Gene-interactions" data-toc-modified-id="Get-TF-Gene-interactions-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Get TF-Gene interactions</a></span></li><li><span><a href="#Get-Sigma-Gene-interactions" data-toc-modified-id="Get-Sigma-Gene-interactions-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Get Sigma-Gene interactions</a></span></li><li><span><a href="#Add-smallRNA-regulation" data-toc-modified-id="Add-smallRNA-regulation-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Add smallRNA regulation</a></span></li><li><span><a href="#Put-it-all-together" data-toc-modified-id="Put-it-all-together-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Put it all together</a></span></li><li><span><a href="#Combine-effects-when-duplicated" data-toc-modified-id="Combine-effects-when-duplicated-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Combine effects when duplicated</a></span></li></ul></li><li><span><a href="#Ecocyc" data-toc-modified-id="Ecocyc-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Ecocyc</a></span><ul class="toc-item"><li><span><a href="#Load-dataset" data-toc-modified-id="Load-dataset-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load dataset</a></span></li><li><span><a href="#Capitalize-first-letter-of-regulator" data-toc-modified-id="Capitalize-first-letter-of-regulator-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Capitalize first letter of regulator</a></span></li><li><span><a href="#Rename-a-few-regulators" data-toc-modified-id="Rename-a-few-regulators-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Rename a few regulators</a></span></li><li><span><a href="#Format-similar-to-RegulonDB" data-toc-modified-id="Format-similar-to-RegulonDB-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Format similar to RegulonDB</a></span></li></ul></li><li><span><a href="#Add-Additional-regulators-that-were-missed-in-the-full-TRN" data-toc-modified-id="Add-Additional-regulators-that-were-missed-in-the-full-TRN-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Add Additional regulators that were missed in the full TRN</a></span><ul class="toc-item"><li><span><a href="#OmpR" data-toc-modified-id="OmpR-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>OmpR</a></span></li><li><span><a href="#YieP" data-toc-modified-id="YieP-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>YieP</a></span></li><li><span><a href="#YgbI" data-toc-modified-id="YgbI-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>YgbI</a></span></li><li><span><a href="#ppGpp" data-toc-modified-id="ppGpp-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>ppGpp</a></span></li></ul></li><li><span><a href="#Regulators-from-Ye-Gao's-yTF-ChIP-exo-Studies" data-toc-modified-id="Regulators-from-Ye-Gao's-yTF-ChIP-exo-Studies-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Regulators from Ye Gao's yTF ChIP-exo Studies</a></span><ul class="toc-item"><li><span><a href="#Remove-YbaO-(same-as-DecR,-which-EcoCyc-has)" data-toc-modified-id="Remove-YbaO-(same-as-DecR,-which-EcoCyc-has)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Remove YbaO (same as DecR, which EcoCyc has)</a></span></li></ul></li><li><span><a href="#Merge-the-TRNs" data-toc-modified-id="Merge-the-TRNs-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Merge the TRNs</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import re
import os

# RegulonDB

In [2]:
def read_regulon_file(filename):
    # Read file
    with open(filename,'r') as f:
        lines = f.readlines()
    
    # Get table start position
    comment = True
    i = 0
    while comment:
        if lines[i].startswith('#'):
            i += 1
        else:
            comment=False
            
#     # Get column names       
#     names = [line[5:-1] for line in lines if re.match('# \d',line)]

    return pd.read_csv(filename,index_col=None,skiprows=i,sep='\t',header=None,#names=names,
                       quoting=3)

## Convert RegulonDB names to b-numbers

In [3]:
DF_gene_info = pd.read_csv('../precise2/gene_info.csv',index_col=0)

In [4]:
DF_genes = read_regulon_file('GeneProductSet.txt')
DF_genes.columns = ['regdb_id','name','bnumber','left','right','strand','product','ev','pmid','evidence','bnumbers','dbxref']

In [5]:
# Check that all genes in the gene table have b-numbers
DF_gene_info[~DF_gene_info.index.isin(DF_genes.bnumber)]

,gene_name,old_locus_tag,start,end,strand,gene_product,COG,uniprot
locus_tag,,,,,,,,
b4053,alr,NaN,4265782,4266861,+,alanine racemase 1,Amino acid transport and metabolism,P0A6B4


In [6]:
name2num = DF_genes[['name','bnumber']].set_index('name')['bnumber'].to_dict()
name2num['alr'] = 'b4053' # Missing in regulonDB bnumber column

## Get TF-Gene interactions

In [7]:
DF_TF_gene = read_regulon_file('network_tf_gene.txt')
DF_TF_gene.columns = ['Regulator','gene_name','effect','ev','evidence']
DF_TF_gene.drop('ev',axis=1,inplace=True)

# Rename some regulators
DF_TF_gene.Regulator.replace({'Dan':'TtdR',
                              'MatA':'EcpR',
                              'YiaJ':'PlaR',
                             'FNR':'Fnr',
                             'CRP':'Crp'},inplace=True)
DF_TF_gene['source'] = 'RegulonDB_TF'

## Get Sigma-Gene interactions

In [8]:
DF_sigma = read_regulon_file('network_sigma_gene.txt')
DF_sigma.columns = ['Regulator','gene_name','effect','ev','evidence']
DF_sigma.drop('evidence',axis=1,inplace=True)

# Split up commas in regulator (e.g. Sigma24,Sigma70)
DF_sigma = DF_sigma.assign(Regulator=DF_sigma.Regulator.str.split(', ')).explode('Regulator')

# Replace Sigma with gene names
DF_sigma.Regulator.replace({'Sigma19':'FecI',
                            'Sigma24':'RpoE',
                            'Sigma28':'FliA',
                            'Sigma32':'RpoH',
                            'Sigma38':'RpoS',
                            'Sigma54':'RpoN',
                            'Sigma70':'RpoD'},inplace=True)
DF_sigma['source'] = 'RegulonDB_sigma'

In [9]:
DF_sigma.head()

,Regulator,gene_name,effect,ev,source
0,RpoE,ahpF,activator,"[AIPP, TIM]",RegulonDB_sigma
1,RpoE,apaG,activator,"[NAS, TASES]",RegulonDB_sigma
2,RpoE,apaH,activator,"[NAS, TASES]",RegulonDB_sigma
3,RpoE,bacA,activator,"[AIPP, TIM]",RegulonDB_sigma
4,RpoE,bamA,activator,"[AIPP, HIPP, RS-EPT-CBR, TIM]",RegulonDB_sigma


## Add smallRNA regulation

In [10]:
DF_srna = read_regulon_file('sRNABindingSiteSet.txt')
DF_srna.columns = ['regdb_id','Regulator','gene_name','left','right','strand','effect','seq','type','mechanism','ev','evidence']
DF_srna = DF_srna[['Regulator','gene_name','effect','evidence']]
DF_srna['source'] = 'RegulonDB_sRNA'

In [11]:
DF_srna.head()

,Regulator,gene_name,effect,evidence,source
0,ryhB,sodB,-,NaN,RegulonDB_sRNA
1,ryhB,uof,-,Weak,RegulonDB_sRNA
2,ryhB,sdhD,-,Weak,RegulonDB_sRNA
3,ryhB,bfr,-,NaN,RegulonDB_sRNA
4,ryhB,fumA,-,Weak,RegulonDB_sRNA


## Put it all together

In [12]:
DF_combined = pd.concat([DF_TF_gene,DF_sigma,DF_srna])

# Replace effects with symbols
DF_combined.effect.replace({'repressor':'-','activator':'+','unknown':'?',None:'?'},inplace=True)

# Replace evidence with numbers
DF_combined.evidence.replace({'Weak':0,'Strong':1,'Confirmed':2},inplace=True)

# Fill empty evidence with Weak
DF_combined.evidence.fillna(0,inplace=True)

# Add b-numbers and remove genes without a b-number (cryptic genes)
DF_combined['bnumber'] = DF_combined.gene_name.replace(name2num)
DF_combined.bnumber.replace({'None':np.nan},inplace=True)
DF_combined = DF_combined[DF_combined.bnumber.notnull()]

In [13]:
DF_combined.head()

,Regulator,gene_name,effect,evidence,source,ev,bnumber
0,AccB,accB,-,0.0,RegulonDB_TF,NaN,b3255
1,AccB,accC,-,0.0,RegulonDB_TF,NaN,b3256
2,AcrR,acrA,-,1.0,RegulonDB_TF,NaN,b0463
3,AcrR,acrB,-,1.0,RegulonDB_TF,NaN,b0462
4,AcrR,acrR,-,1.0,RegulonDB_TF,NaN,b0464


## Combine effects when duplicated

In [14]:
list2struct = []
for (reg,gene,bnum,source),group in DF_combined.groupby(['Regulator','gene_name','bnumber','source']):
    evidence = max(group.evidence)
    effect = ''.join(set(group.effect))
    if effect == '-+' or effect == '+-':
        effect = '?'
    elif effect == '?-':
        effect = '-'
    elif effect == '-?':
        effect = '-'
    elif effect == '?+':
        effect = '+'
    elif effect == '+?':
        effect = '+'
    list2struct.append([reg,gene,bnum,effect,evidence,source])

DF_regulonDB = pd.DataFrame(list2struct,columns=['regulator','gene_name','gene_id','effect','evidence','source'])

# Ensure that all interactions are unique
assert(len(DF_regulonDB) == len(DF_regulonDB.drop_duplicates(['regulator','gene_id'])))

# Ecocyc

## Load dataset

In [15]:
DF_raw_ecocyc = pd.read_csv('ecocyc_regulator_table.txt',sep='\t')
DF_raw_ecocyc.columns = ['ecocyc_tf','full_regulator','ecocyc_reg','regulator','ecocyc_genes','gene_name','gene_id']
DF_raw_ecocyc = DF_raw_ecocyc[['full_regulator','regulator','gene_name','gene_id']]

# Add non-gene regulators
DF_raw_ecocyc.regulator = DF_raw_ecocyc.regulator.fillna(DF_raw_ecocyc.full_regulator)

In [16]:
DF_raw_ecocyc.head()

,full_regulator,regulator,gene_name,gene_id
0,30S ribosomal subunit protein S1,rpsA,rpsA,b0911
1,30S ribosomal subunit protein S15,rpsO,rpsO,b3165
2,30S ribosomal subunit protein S2,rpsB,tsf // rpsB // tff,b0170 // b0169 // b4414
3,30S ribosomal subunit protein S4,rpsD,rplQ // rpsK // rpsD // rpsM,b3294 // b3297 // b3296 // b3298
4,30S ribosomal subunit protein S7,rpsG,fusA // tufA // rpsG // rpsL,b3340 // b3339 // b3341 // b3342


## Capitalize first letter of regulator

In [17]:
# Ignore sRNAs
sRNA_list = DF_srna.Regulator.unique()

def capitalize_proteins(x):
    if x in sRNA_list:
        return x
    else:
        return x[0].upper()+x[1:]

DF_raw_ecocyc.regulator = DF_raw_ecocyc.regulator.apply(capitalize_proteins)

## Rename a few regulators

In [18]:
DF_raw_ecocyc.regulator = DF_raw_ecocyc.regulator.replace({'H<SUP>+</SUP>':'H+',
                                                           'Mn<SUP>2+</SUP>':'Mn2+',
                                                           '<i>S</i>-adenosyl-L-methionine':'SAM',
                                                           'An L-histidyl-[tRNA<SUP>His</SUP>]':'His-tRNA',
                                                           'An L-isoleucyl-[tRNA<SUP>Ile</SUP>]':'Ile-tRNA',
                                                           'An L-leucyl-[tRNA<SUP>Leu</SUP>]':'Leu-tRNA',
                                                           'An L-phenylalanyl-[tRNA<SUP>Phe</SUP>]':'Phe-tRNA',
                                                           'An L-threonyl-[tRNA<SUP>Thr</SUP>]':'Thr-tRNA',
                                                           'An L-tryptophanyl-[tRNA<SUP>Trp</SUP>]':'Trp-tRNA',
                                                           'An L-valyl-[tRNA<SUP>Val</SUP>]':'Val-tRNA',
                                                           'Hns':'H-NS',
                                                           'GlnG':'NtrC',
                                                           'PpGpp':'ppGpp',
                                                           'AccA // accD':'AccDA',
                                                           'Sxy // crp':'CRP-Sxy',
                                                           'YafQ // dinJ':'DinJ-YafQ',
                                                           'HupA // hupB':'HU',
                                                           'IhfA // ihfB':'IHF',
                                                           'FlhD // flhC':'FlhDC',
                                                           'RcsB // gadE':'GadE-RcsB',
                                                           'HigA // higB':'HigBA',
                                                           'HipB // hipA':'HipAB',
                                                           'MazF // mazE':'MazE-MazF',
                                                           'RcsB // rcsA':'RcsAB',
                                                           'BglJ // rcsB':'RcsB-BglJ',
                                                           'RelB // relE':'RelB-RelE',
                                                           'RpsR // rpsF':'RpsR-RpsF',
                                                           'SelB // selC':'SelB-Sec-tRNA',
                                                           'YoeB // yefM':'YefM-YoeB'
                                                           })

## Format similar to RegulonDB

In [19]:
list2struct = []
for i,row in DF_raw_ecocyc.iterrows():
    genes = row.gene_name.split(' // ')
    for phantom in ['sroD','raiZ','tpke11','insCD5']:
        if phantom in genes:
            genes.remove(phantom)
        
    bnums = row.gene_id.split(' // ')
    assert(len(genes) == len(bnums))
    for j in range(len(genes)):
        list2struct.append([row.regulator,genes[j],bnums[j]])
DF_ecocyc = pd.DataFrame(list2struct,columns=['regulator','gene_name','gene_id'])

In [20]:
DF_ecocyc = DF_ecocyc[['regulator','gene_name','gene_id']]
DF_ecocyc['evidence'] = None
DF_ecocyc['effect'] = None
DF_ecocyc['source'] = 'Ecocyc'

In [21]:
DF_ecocyc.head()

,regulator,gene_name,gene_id,evidence,effect,source
0,RpsA,rpsA,b0911,None,None,Ecocyc
1,RpsO,rpsO,b3165,None,None,Ecocyc
2,RpsB,tsf,b0170,None,None,Ecocyc
3,RpsB,rpsB,b0169,None,None,Ecocyc
4,RpsB,tff,b4414,None,None,Ecocyc


# Add Additional regulators that were missed in the full TRN

## OmpR

In [22]:
ompR_act = ['ompF','csgD','csgE','csgF','csgG','narU','narZ','narY','narW','narV','dtpA','znuA','ompC','micF']
ompR_rep = ['copA','nmpC','yccT','htpX','yobB','exoX','nuoN','galP','alx','malE','malF','malG']

In [23]:
list2struct = [['OmpR',gene,name2num[gene],1,'+','PMID:28526842'] for gene in ompR_act] + \
             [['OmpR',gene,name2num[gene],1,'-','PMID:28526842'] for gene in ompR_rep]

In [24]:
DF_ompR = pd.DataFrame(list2struct,columns=DF_ecocyc.columns)

## YieP

In [25]:
yieP_rep = ['ykgE','ykgF','ykgG','yohJ','yohK','adiY']
list2struct = [['YieP',gene,name2num[gene],1,'-','PMID:33149261'] for gene in yieP_rep]
DF_yieP = pd.DataFrame(list2struct,columns=DF_ecocyc.columns)

## YgbI

In [26]:
ygbI_rep = ['ygbI','ygbJ','ygbK','ygbL','ygbM','ygbN']
list2struct = [['YgbI',gene,name2num[gene],1,'-','PMID:31797920'] for gene in ygbI_rep]
DF_ygbI = pd.DataFrame(list2struct,columns=DF_ecocyc.columns)

## ppGpp

In [27]:
# Load dataset - Downloaded from Sanchez-Vasquez et al 2019
DF_ppGpp = pd.read_excel('dataset_s01_from_sanchez_vasquez_2019.xlsx',engine='openpyxl',sheet_name='Data')

# Get 757 genes described to be directly regulated by ppGpp
ppGpp_act = DF_ppGpp[DF_ppGpp['1+2+ 5 min Category'] == 'A'].Synonym.values
ppGpp_rep = DF_ppGpp[DF_ppGpp['1+2+ 5 min Category'] == 'B'].Synonym.values
len(ppGpp_act)+len(ppGpp_rep)

757

In [28]:
num2name = {v:k for k,v in name2num.items()}

In [29]:
list2struct = [['ppGpp',num2name[gene],gene,1,'+','PMID:30971496'] for gene in ppGpp_act] + \
             [['ppGpp',num2name[gene],gene,1,'-','PMID:30971496'] for gene in ppGpp_rep]

In [30]:
DF_ppGpp = pd.DataFrame(list2struct,columns=DF_ecocyc.columns)

In [31]:
DF_ppGpp

,regulator,gene_name,gene_id,evidence,effect,source
0,ppGpp,ydgI,b1605,1,+,PMID:30971496
1,ppGpp,mgtA,b4242,1,+,PMID:30971496
2,ppGpp,deaD,b3162,1,+,PMID:30971496
3,ppGpp,lysP,b2156,1,+,PMID:30971496
4,ppGpp,ybiT,b0820,1,+,PMID:30971496
...,...,...,...,...,...,...
752,ppGpp,eutD,b2458,1,-,PMID:30971496
753,ppGpp,gltA,b0720,1,-,PMID:30971496
754,ppGpp,katG,b3942,1,-,PMID:30971496
755,ppGpp,yafD,b0209,1,-,PMID:30971496


# Regulators from Ye Gao's yTF ChIP-exo Studies

In [32]:
# re-format and re-name columns
DF_gao_s4 = pd.read_excel('gao_dataset_s4.xlsx', engine='openpyxl', sheet_name='pre_formatted')
DF_gao_s4 = DF_gao_s4.drop(columns=['Start', 'End', 'S/N']).rename(
    columns={'b_number': 'gene_id', 'protein': 'regulator'})
DF_gao_s4['gene_name'] = DF_gao_s4['gene_id'].apply(lambda gene_id: num2name.get(gene_id, None))
DF_gao_s4['evidence'] = 0
DF_gao_s4['effect'] = '?'
DF_gao_s4['source'] = 'Gao2021'
DF_gao_s4.head()

,gene_id,regulator,gene_name,evidence,effect,source
0,b0088,YidZ,murD,0,?,Gao2021
1,b0134,YidZ,panB,0,?,Gao2021
2,b0189,YidZ,rof,0,?,Gao2021
3,b0282,YidZ,yagP,0,?,Gao2021
4,b0344,YidZ,lacZ,0,?,Gao2021


In [33]:
# re-format and re-name columns
DF_gao_s5 = pd.read_excel('gao_dataset_s5.xlsx', engine='openpyxl', sheet_name='pre_formatted')
DF_gao_s5 = DF_gao_s5.drop(columns=['Transcription Unit']).rename(
    columns={'b_number': 'gene_id', 'protein': 'regulator'})
DF_gao_s5['gene_name'] = DF_gao_s5['gene_id'].apply(lambda gene_id: num2name.get(gene_id, None))
DF_gao_s5['evidence'] = 0
DF_gao_s5['effect'] = '?'
DF_gao_s5['source'] = 'Gao2021'
DF_gao_s5.head()

,gene_id,regulator,gene_name,evidence,effect,source
0,b0065,YeiE,yabI,0,?,Gao2021
1,b0072,YeiE,leuC,0,?,Gao2021
2,b0077,YeiE,ilvI,0,?,Gao2021
3,b0343,YeiE,lacY,0,?,Gao2021
4,b0411,YeiE,tsx,0,?,Gao2021


## Remove YbaO (same as DecR, which EcoCyc has)

In [35]:
DF_gao_s5 = DF_gao_s5[DF_gao_s5['regulator'] != 'YbaO']

# Merge the TRNs

In [36]:
DF_TRN = pd.concat([DF_regulonDB,DF_ecocyc,DF_ompR,DF_yieP,DF_ygbI,DF_ppGpp,DF_gao_s4,DF_gao_s5]).reset_index(drop=True)

# Sort by Evidence level and remove duplicate regulation
DF_TRN = DF_TRN.sort_values('evidence').drop_duplicates(['regulator','gene_name','gene_id'])
DF_TRN = DF_TRN.reset_index(drop=True)

In [37]:
DF_TRN.describe()

,regulator,gene_name,gene_id,effect,evidence,source
count,10795,10789,10795,8799,8799.0,10795
unique,371,3411,3404,3,3.0,9
top,RpoD,csgE,b1037,+,0.0,RegulonDB_TF
freq,1698,25,25,5256,5377.0,4426


In [38]:
DF_TRN.to_csv('TRN.csv',index=False)